# Web Scraping and RAG Together

In [1]:
from BRAD import brad
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb
import subprocess
import logging
import os

from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# What is a Single Cell Foundation Model?

## Initialize the Empty Database

In [2]:
docsPath='papers/'
dbName='database/'
dbPath='databases/'
HuggingFaceEmbeddingsModel = 'BAAI/bge-base-en-v1.5'
chunk_size=[700]
chunk_overlap=[200]
v=True

In [3]:
dbPath   += dbName

local = os.getcwd()  ## Get local dir
os.chdir(local)      ## shift the work dir to local dir
print('\nWork Directory: {}'.format(local)) if v else None

#%% Phase 1 - Load DB
embeddings_model = HuggingFaceEmbeddings(model_name=HuggingFaceEmbeddingsModel)
print("\nDocuments loading from: 'str(docsPath)") if v else None
text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader(docsPath,
                         glob="**/*.pdf",
                         loader_cls=PyPDFLoader,
                         #loader_kwargs=text_loader_kwargs,
                         show_progress=True,
                         use_multithreading=True)
docs_data = loader.load()
print('\nDocuments loaded...') if v else None

for i in range(len(chunk_size)):
    for j in range(len(chunk_overlap)):
        text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size[i],
                                                        chunk_overlap = chunk_overlap[j],
                                                        separators=[" ", ",", "\n", ". "])
        data_splits = text_splitter.split_documents(docs_data)
        print("Documents split into chunks...") if v else None
        print("Initializing Chroma Database...") if v else None

        dbName = "DB_cosine_cSize_%d_cOver_%d" %(chunk_size[i], chunk_overlap[j])

        p2_2 = subprocess.run('mkdir  %s/*'%(dbPath+dbName), shell=True)
        _client_settings = chromadb.PersistentClient(path=(dbPath+dbName))

        vectordb = Chroma.from_documents(documents           = data_splits,
                                         embedding           = embeddings_model,
                                         client              = _client_settings,
                                         collection_name     = dbName,
                                         collection_metadata = {"hnsw:space": "cosine"})
print("Database built")


Work Directory: /home/jpic/RAG-DEV/tutorials/scrape-rag


/home/jpic/.local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/jpic/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Documents loading from: 'str(docsPath)


100%|██████████| 1/1 [00:00<00:00, 21.49it/s]


Documents loaded...
Documents split into chunks...
Initializing Chroma Database...



mkdir: cannot create directory ‘databases/database/DB_cosine_cSize_700_cOver_200/*’: No such file or directory


Database built


## Apply BRAD

In [4]:
brad.chat(ragvectordb=vectordb, max_api_calls=8)

Enter your NVIDIA API key:  ········


2024-07-22 13:26:44 INFO semantic_router.utils.logger local


Welcome to RAG! The chat log from this conversation will be saved to /home/jpic/BRAD/2024-07-22_13-26-43/log.json. How can I help?


Input >>  /force PLANNER The process should have 3 stages: (1) Use the RAG to answer the questions: "What are single cell foundation models?" (2) Use arXiv (no other database) to find new research related to this question. (3) Decide if the answer from Stage (1) is good and the pipeline can exit or if the pipeline should return to Step (1) with the updated literature.  Do not use the module name outside of the Step X method: module name and do not add a final stage for writing.


2024-07-22 13:26:51,378 - INFO - PLANNER


...



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Source: None
Content: Single cell foundation models are large language models used in the analysis of individual cells within tissues and organisms. They help identify and recognize various cell types, subpopulations, and rare cell states that may have been previously concealed in bulk measurements. Specific downstream tasks in single-cell analysis include identifying cell types and states, discovering novel cell populations, inferring gene regulation networks, and more. These models have found successful applications in genomics, transcriptomics, proteomics, and drug discovery.

Source: None
Content: Single-cell foundational models are tools that accurately categorize known cell types, as indicated 

2024-07-22 14:51:22,934 - INFO - 


route



2024-07-22 14:51:22,934 - INFO - RAG
2024-07-22 14:51:22,935 - INFO - 

PLANNER:

{'module': 'RAG', 'steps': [{'func': 'rag.retrieval', 'multiquery': False, 'similarity': True, 'mmr': True, 'num-docs': 40, 'docs': "[Document(page_content='ability to uncover cellular heterogeneity within tissues and organisms. It facilitates the \\nidentification and recognition of diverse cell types, subpopulations, and rare cell states that were \\npreviously concealed in bulk measurements. As we discussed  in the previous sections, large \\nlanguage models have found successful applications in the do mains of  genomics, transcriptomics, \\nproteomics and drug discovery. In this section, our attention turns to single -cell language models \\nthat can be employed for various downstream tasks in single -cell analysis. These tasks include \\nidentifying cell types and  states, discovering novel cell populations, inferring gene regulation \\nnetworks, and'), Do


> Finished chain.

> Finished chain.
Single cell foundation models are artificial intelligence models designed to analyze and understand data from individual cells. They are constructed using single-cell transcriptomic data, which measures the expression levels of genes in individual cells. These models utilize machine learning algorithms, such as transformers, to learn gene-level and cell-level representation. They can accurately categorize known cell types and have the potential to uncover new biological insights in single-cell RNA sequencing data. Single cell foundation models are used in genomics and multi-omics research, showing promise in various downstream tasks, such as identifying cell types and states, discovering novel cell populations, inferring gene regulation networks, and more. Examples of single cell foundation models include scGPT, Genefomer, and scFoundation. They are a growing field of study, with ongoing advancements expected as single-cell technologies and omics s

Input >>  exit


Thanks for chatting today! I hope to talk soon, and don't forget that a record of this conversation is available at: /home/jpic/BRAD/2024-07-22_13-26-43/log.json


## Construct a Table of all the Downloaded/Searched Articles

In [7]:
import pandas as pd
import os

# Define the directory
directory = '/home/jpic/BRAD/2024-07-22_13-26-43/'

# Load all CSV files in the directory
csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

# Read and concatenate all CSV files into a single dataframe
dataframes = [pd.read_csv(os.path.join(directory, f)) for f in csv_files]
# combined_df = pd.concat(dataframes, ignore_index=True)
for i, df in enumerate(dataframes):
    dataframes[i] = df[['Title', 'Authors']]
# combined_df = combined_df[['Title', 'Authors']]

    # Convert the dataframe to a LaTeX table
    latex_table = dataframes[i].to_latex(index=False)
    latex_table = latex_table.replace('\\toprule', '\hline'). replace(r'\\', r'\\\hline').replace('\midrule', '\hline')
# Save the LaTeX table to a file
# with open('combined_table.tex', 'w') as f:
#     f.write(latex_table)

# print("LaTeX table saved to combined_table.tex")
    print(csv_files[i])
    print(latex_table)

S3-arxiv-search-machine learning for single cell analysis.csv
\begin{tabular}{ll}
\hline
Title & Authors \\\hline
\hline
SeqMate: A Novel Large Language Model Pipeline for Automating RNA Sequencing & Devam Mondal,Atharva Inamdar \\\hline
Enhancing Fluorescence Correlation Spectroscopy withMachineLearningfor AdvancedAnalysisof Anomalous Diffusion & Nathan Quiblier,Jan-Michael Rye,Pierre Leclerc,Henri Truong,Abdelkrim Hannou,Laurent Héliot,Hugues Berry \\\hline
Enhanced Gene Selection inSingle-CellGenomics: Pre-Filtering Synergy and Reinforced Optimization & Weiliang Zhang,Zhen Meng,Dongjie Wang,Min Wu,Kunpeng Liu,Yuanchun Zhou,Meng Xiao \\\hline
Advances inMachineLearning, Statistical Methods, and AI forSingle-CellRNA Annotation Using Raw Count Matrices in scRNA-seq Data & Megha Patel,Nimish Magre,Himanshi Motwani,Nik Bear Brown \\\hline
Lithium-Ion Battery System Health Monitoring and FaultAnalysisfrom Field Data Using Gaussian Processes & Joachim Schaeffer,Eric Lenz,Duncan Gulla,Marti

# What is the role of MYOD for Cellular Reprogramming

## Initilaize an Empty Database

In [3]:
docsPath='papers/'
dbName='database/'
dbPath='databases/'
HuggingFaceEmbeddingsModel = 'BAAI/bge-base-en-v1.5'
chunk_size=[700]
chunk_overlap=[200]
v=True

In [4]:
dbPath   += dbName

local = os.getcwd()  ## Get local dir
os.chdir(local)      ## shift the work dir to local dir
print('\nWork Directory: {}'.format(local)) if v else None

#%% Phase 1 - Load DB
embeddings_model = HuggingFaceEmbeddings(model_name=HuggingFaceEmbeddingsModel)
print("\nDocuments loading from: 'str(docsPath)") if v else None
text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader(docsPath,
                         glob="**/*.pdf",
                         loader_cls=PyPDFLoader,
                         #loader_kwargs=text_loader_kwargs,
                         show_progress=True,
                         use_multithreading=True)
docs_data = loader.load()
print('\nDocuments loaded...') if v else None

for i in range(len(chunk_size)):
    for j in range(len(chunk_overlap)):
        text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size[i],
                                                        chunk_overlap = chunk_overlap[j],
                                                        separators=[" ", ",", "\n", ". "])
        data_splits = text_splitter.split_documents(docs_data)
        print("Documents split into chunks...") if v else None
        print("Initializing Chroma Database...") if v else None

        dbName = "DB_cosine_cSize_%d_cOver_%d" %(chunk_size[i], chunk_overlap[j])

        p2_2 = subprocess.run('mkdir  %s/*'%(dbPath+dbName), shell=True)
        _client_settings = chromadb.PersistentClient(path=(dbPath+dbName))

        vectordb = Chroma.from_documents(documents           = data_splits,
                                         embedding           = embeddings_model,
                                         client              = _client_settings,
                                         collection_name     = dbName,
                                         collection_metadata = {"hnsw:space": "cosine"})
print("Database built")


Work Directory: /home/jpic/RAG-DEV/tutorials/scrape-rag


/home/jpic/.local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/jpic/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Documents loading from: 'str(docsPath)


100%|██████████| 1/1 [00:00<00:00, 12.66it/s]


Documents loaded...
Documents split into chunks...
Initializing Chroma Database...



mkdir: cannot create directory ‘databases/database/DB_cosine_cSize_700_cOver_200/*’: No such file or directory


Database built


## Apply BRAD

In [ ]:
brad.chat()

Enter your NVIDIA API key:  ········



Would you like to use a database with BRAD [Y/N]?


 N


2024-07-22 15:57:09 INFO semantic_router.utils.logger local


Welcome to RAG! The chat log from this conversation will be saved to /home/jpic/BRAD/2024-07-22_15-56-59/log.json. How can I help?


...

BRAD >> 1: 


> Entering new ConversationChain chain...
Prompt after formatting:
Current conversation:
Human: what is MYOD in reprogramming on pubmed?
BRAD: Database: PUBMED
Search Terms: MYOD, gene reprogramming, cell fate reprogramming, pluripotency, transcription factor, induced pluripotent stem cells, reprogramming factors, direct reprogramming, somatic cell reprogramming, muscle cell reprogramming

Query:What is a single cell foundation model? Please base this off of PubMed

From the query, decide if ARXIV, PUBMED, or BIORXIV should be searched, and propose no more than 10 search terms for this query and database. Separate each term with a comma, and provide no extra information/explination for either the database or search terms. Do not include general purpose search terms such as "recent advances", "novel", "current research", "latest trends", "theoretical foundations", or other general query terms. Also, do not use very general fields as serch terms such as: "dynamical systems"

2024-07-22 15:58:55,425 - INFO - {'database': 'PUBMED', 'search_terms': ['single cell foundation models', 'single cell transcriptional networks', 'foundation models for single cell analysis', 'single cell gene regulation', 'cellular heterogeneity', 'single cell atlas', 'scRNA-seq foundation models', 'single cell model building', 'cell type classification', 'biological single cell modeling']}
2024-07-22 15:58:55,425 - INFO - searching on PUBMED...
2024-07-22 15:58:55,425 - INFO - Search Terms: ['single cell foundation models', 'single cell transcriptional networks', 'foundation models for single cell analysis', 'single cell gene regulation', 'cellular heterogeneity', 'single cell atlas', 'scRNA-seq foundation models', 'single cell model building', 'cell type classification', 'biological single cell modeling']
2024-07-22 15:58:55,426 - INFO - search_pubmed_article
2024-07-22 15:58:55,426 - INFO - term=single cell foundation models



> Finished chain.


2024-07-22 15:58:55,771 - INFO - handle=<http.client.HTTPResponse object at 0x151057af4bb0>
2024-07-22 15:58:55,775 - INFO - record={'Count': '6547', 'RetMax': '10', 'RetStart': '0', 'IdList': ['31597160', '35649411', '35063085', '32415072', '37289589', '33388438', '32344649', '27716510', '37149868', '30918367'], 'TranslationSet': [{'From': 'single', 'To': '"single person"[MeSH Terms] OR ("single"[All Fields] AND "person"[All Fields]) OR "single person"[All Fields] OR "single"[All Fields] OR "singles"[All Fields]'}, {'From': 'cell', 'To': '"cells"[MeSH Terms] OR "cells"[All Fields] OR "cell"[All Fields]'}, {'From': 'foundation', 'To': '"foundation\'s"[All Fields] OR "foundational"[All Fields] OR "foundations"[MeSH Terms] OR "foundations"[All Fields] OR "foundation"[All Fields]'}, {'From': 'models', 'To': '"model"[All Fields] OR "model\'s"[All Fields] OR "modeled"[All Fields] OR "modeler"[All Fields] OR "modeler\'s"[All Fields] OR "modelers"[All Fields] OR "modeling"[All Fields] OR "mod


New documents loaded...
New document split into chunks...


# Scratch

## Build the Database More

Here is the outlined pipeline:

**Step 1 (RAG)**:
Prompt: Use RAG to search literature and documents from a text database to understand the role of PCNA in the cell cycle and the pip Fucci system. Provide a summary of the findings.

**Step 2 (ROUTER)**:
Prompt: Evaluate the quality of the results from Step 1. If the results are satisfactory, proceed to Step 5. Otherwise, proceed to Step 3.

**Step 3 (SCRAPE)**:
Prompt: Search bioRxiv for recent research articles related to the cell cycle. Use the knowledge gained from Step 1 to inform the search query.

**Step 4 (ROUTER)**:
Prompt: Regardless of the outcome, always return to Step 1 to refine or expand the search.

**Step 5 (WRITE)**:
Prompt: Synthesize and summarize the entire analysis process, including the findings from Step 1, any additional insights gained from Step 3, and the iterative process of refinement. Provide a final report that encapsulates the entire analysis.

Note: The ROUTER steps are used to control the flow of the pipeline, and the WRITE step is used to summarize the results and create a final report. The dependencies between steps are indicated in the prompts.


Do you want to proceed with this plan? [Y/N/edit]

In [ ]:
brad.chat(ragvectordb=vectordb)

2024-07-21 11:11:00 INFO semantic_router.utils.logger local


Welcome to RAG! The chat log from this conversation will be saved to /home/jpic/BRAD/2024-07-21_11-11-00/log.json. How can I help?


Input >>  /force PLANNER make a plan to iterate between RAG and SCRAPE until the following question can be answered by the RAG well: what is the core of a hypergraph. The process should have 3 stages: (1) use the RAG to answer the question. (2) use arXiv (no other database) to find new research related to this question. Use the search term "Hypergraph Core" (3) Decide if the answer from (1) is good and the system can exit or if the system should return to step (1) with the updated literature. Do not use the module name outside of the Step X method: module name


2024-07-21 11:11:08,332 - INFO - PLANNER


BRAD >> 0: 


> Entering new ConversationChain chain...
Prompt after formatting:
**INSTRUCTIONS:**
You are planning a bioinformatics analysis pipeline to address a user's query. Your task is to outline a multi-step workflow using the available methods listed below. Each method should be used appropriately to ensure a thorough analysis. For each step in the pipeline, explain the goal of the process as well as provide a prompt to the chatbot that will execute that step. If information is passed between stages, such as from literature or code to databases, indicate the dependencies of steps in the prompt.

**Available Methods:**
1. **RAG**: Look up literature and documents from a text database.
2. **SCRAPE**: Search platforms like arXiv, bioRxiv, and PubMed for the latest research.
3. **DATABASE**: Utilize bioinformatics databases such as Gene Ontology and Enrichr to perform gene set enrichment analyses.
4. **CODE**: Execute bioinformatics pipelines. Utilize prebuilt pipelines or develop 

...

,Title,Authors,Abstract
0,HeterogeneousHypergraphEmbedding for Recommend...,"Darnbi Sakong,Viet Hung Vu,Thanh Trung Huynh,P...",Recent advancements in recommender systems hav...
1,Combinatorics on Social Configurations,"Dylan Laplace Mermoud,Pierre Popoli",…the trivial set system of all players is one ...
2,Retrosynthesis Prediction via Search in (Hyper...,"Zixun Lan,Binjie Hong,Jiajun Zhu,Zuo Zeng,Zhen...",Predicting reactants from a specifiedcoreprodu...
3,The structural evolution of temporalhypergraph...,"Marco Mancastroppa,Iacopo Iacopini,Giovanni Pe...",…and their temporal dynamics constitute crucia...
4,CURSOR: Scalable Mixed-OrderHypergraphMatching...,"Qixuan Zheng,Ming Zhang,Hong Yan","To achieve greater accuracy,hypergraphmatching..."
5,Faster Combinatorial k-Clique Algorithms,"Amir Abboud,Nick Fischer,Yarin Shechter",…-Clique to Triangle detection that exploits t...
6,Exploring Cohesive Subgraphs inHypergraphs: Th...,"Dahee Kim,Junghoon Kim,Sungsu Lim,Hyun Ji Jeong",Identifying cohesive subgraphs inhypergraphsis...
7,Stability and Generalization ofHypergraphColla...,"Michael Ng,Hanrui Wu,Andy Yip",…shown to be very effective in utilizing pairw...
8,Thinking Like an Expert:MultimodalHypergraph-o...,"Fanglong Yao,Changyuan Tian,Jintao Liu,Zequn Z...",…expert owns two prominent characteristics tha...
9,ScalableHypergraphVisualization,"Peter Oliver,Eugene Zhang,Yue Zhang",Hypergraphvisualization has many applications ...


2024-07-21 11:11:17,691 - INFO - Directory '/home/jpic/BRAD/2024-07-21_11-11-00/pdf' created successfully
2024-07-21 11:11:55,560 - INFO - https://arxiv.org/search/?searchtype=all&query=Hypergraph+Theory&abstracts=show&size=50&order=
2024-07-21 11:11:55,561 - INFO - Directory '/home/jpic/RAG-DEV/tutorials/scrape-rag/arxiv' created successfully


searching the following on arxiv: Hypergraph Theory


,Title,Authors,Abstract
0,On the number of sets with small sumset,"Dingyuan Liu,Letícia Mattos,Tibor Szabó","…sumsets, improving results by Campos, Coulson..."
1,On the complexity of Client-Waiter and Waiter-...,"Valentin Gledel,Nacim Oijid,Sébastien Tavenas,...",Positional games were introduced by Hales and ...
2,The convergence and uniqueness of a discrete-t...,"Ruowei Li,Florentin Münch","…-Laplace operators. Moreover, our results can..."
3,Competition between group interactions and non...,"Jihye Kim,Deok-Sun Lee,Byungjoon Min,Mason A. ...",Social dynamics are often driven by both pairw...
4,The bunkbed conjecture is not robust to genera...,Lawrence Hollom,"The bunkbed conjecture, which has featured in ..."
5,"Weak recovery, hypothesis testing, and mutual ...","Elchanan Mossel,Allan Sly,Youngtak Sohn",…the observed network and the community struct...
6,Combinatorics on Social Configurations,"Dylan Laplace Mermoud,Pierre Popoli","In cooperative gametheory, the social configur..."
7,Collective Bit Flipping-Based Decoding of Quan...,"Dimitris Chytas,Nithin Raveendran,Bane Vasić",…of variable degree-3 (dv-3) QLDPC codes under...
8,Intersecting sets in probability spaces and Sh...,"Artem Chernikov,Henry Towsner",…are indexed by multiple indices. In particula...
9,EquivalenceHypergraphs: E-Graphs for MonoidalT...,"Dan R. Ghica,Chris Barrett,Aleksei Tiurin","…graphs with an equivalence relation, called e..."


2024-07-21 11:11:56,072 - INFO - Directory '/home/jpic/BRAD/2024-07-21_11-11-00/pdf' created successfully
 58%|█████▊    | 14/24 [00:14<00:12,  1.29s/it]

In [4]:
vectordb

In [ ]:
/force PLANNER make a plan with the following stages: 1. RAG, with prompt: What is the PCNA in the cell cycle and pip fucci system, 2. If the router answer is good, skip to step 5 else go to step 3. Step 3: SCRAPE PubMed for cell cycle articles. Step 4: return to step 1. Step 5: Write a summary of the process. STEP 4 should be ROUTER that always returns to step 1